In [1]:

import numpy as np
import pandas as pd

matrix = np.array([
    [0.181, 0.074, 0.067, 0.101],
    [0.078, 0.055, 0.049, 0.093],
    [0.021, 0.069, 0.035, 0.09],
    [0.033, 0.078, 0.076, 0.08],
    [0.028, 0.081, 0.235, 0.071],
    [0.304, 0.052, 0.112, 0.062],
    [0.038, 0.143, 0.032, 0.131],
    [0.014, 0.095, 0.045, 0.134],
    [0.028, 0.144, 0.136, 0.095],
    [0.014, 0.076, 0.148, 0.081],
    [0.009, 0.132, 0.065, 0.063]
])

criteria = ['C1', 'C2', 'C3', 'C4']
sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10', 'Site11']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.16376103, 0.29605439, 0.46393757, 0.076247])

# V-shape preference thresholds (can be tuned)
p_list = [2.998, 1.441, 0.783, 3.957]

def normalize_matrix(matrix, criteria_types):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        if criteria_types[j] == 'max':
            norm_matrix[:, j] = (col - np.min(col)) / (np.max(col) - np.min(col))
        else:  # 'min'
            norm_matrix[:, j] = (np.max(col) - col) / (np.max(col) - np.min(col))
    return norm_matrix

def vshape_preference(d, p):
    return np.clip(d / p, 0, 1)

def calculate_preference_matrix(norm_matrix, criteria_types, weights, p_list):
    n = norm_matrix.shape[0]
    m = norm_matrix.shape[1]
    preference_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            pref_sum = 0
            for k in range(m):
                if criteria_types[k] == 'max':
                    diff = norm_matrix[i, k] - norm_matrix[j, k]
                else:  # 'min'
                    diff = norm_matrix[j, k] - norm_matrix[i, k]
                pref = vshape_preference(diff, p_list[k])
                pref_sum += weights[k] * pref
            preference_matrix[i, j] = pref_sum
    return preference_matrix

def calculate_flows(preference_matrix):
    n = preference_matrix.shape[0]
    phi_plus = preference_matrix.sum(axis=1) / (n - 1)
    phi_minus = preference_matrix.sum(axis=0) / (n - 1)
    net_flows = phi_plus - phi_minus
    return phi_plus, phi_minus, net_flows

def promethee_ii(matrix, criteria_types, weights, p_list, sites, criteria):
    norm_matrix = normalize_matrix(matrix, criteria_types)
    preference_matrix = calculate_preference_matrix(norm_matrix, criteria_types, weights, p_list)
    phi_plus, phi_minus, net_flows = calculate_flows(preference_matrix)
    
    # Prepare results DataFrame
    df = pd.DataFrame(matrix, columns=criteria, index=sites)
    df['Leaving Flow'] = phi_plus
    df['Entering Flow'] = phi_minus
    df['Net Flow'] = net_flows
    df['Rank'] = df['Net Flow'].rank(ascending=False, method='min').astype(int)
    df = df.sort_values('Rank')
    return df

# Run PROMETHEE II
result_df = promethee_ii(matrix, criteria_types, weights, p_list, sites, criteria)
print(result_df)


           C1     C2     C3     C4  Leaving Flow  Entering Flow  Net Flow  \
Site5   0.028  0.081  0.235  0.071      0.434713       0.057920  0.376794   
Site9   0.028  0.144  0.136  0.095      0.312155       0.043713  0.268442   
Site10  0.014  0.076  0.148  0.081      0.222129       0.089173  0.132956   
Site6   0.304  0.052  0.112  0.062      0.169198       0.157307  0.011891   
Site11  0.009  0.132  0.065  0.063      0.135221       0.134279  0.000942   
Site7   0.038  0.143  0.032  0.131      0.142466       0.184897 -0.042431   
Site1   0.181  0.074  0.067  0.101      0.072011       0.161486 -0.089476   
Site4   0.033  0.078  0.076  0.080      0.064601       0.154309 -0.089708   
Site8   0.014  0.095  0.045  0.134      0.059661       0.186064 -0.126403   
Site2   0.078  0.055  0.049  0.093      0.021985       0.234006 -0.212021   
Site3   0.021  0.069  0.035  0.090      0.010769       0.241755 -0.230986   

        Rank  
Site5      1  
Site9      2  
Site10     3  
Site6      4  
